# OpenGradient Modelthon Workflow Guide

This notebook will provide an example workflow of how modelthon participants can train their model, convert it to ONNX format, upload it to the OpenGradient model hub, and execute an inference on the OpenGradient network.

Here, we will demonstrate the following pipeline:
1. Load data using utils functions into your model
2. Train the model using the included training utilities
3. Convert the model to ONNX
4. Verify predictions are the same pre and post ONNX conversion
5. Utilize the `OG-SDK` to upload the model to the OpenGradient network & run a test inference

### Install Dependencies

In [ ]:
!pip install -r requirements.txt

## Run the Example Model

In [1]:
import pandas as pd
import numpy as np
import onnxruntime as ort
import os

from utils.training import train_linear_model

DATA_PATH = "../data/ETHUSDT_1h_spot_forecast_training.csv"
ONNX_PATH = "eth_ridge_regression.onnx"

### 1. Data Loading and Feature Preparation
This cell loads the training data and prepares features using the LinearTimeSeriesPreprocessor

In [2]:
print("=== Loading Data ===")
df = pd.read_csv(DATA_PATH)

feature_cols = []
for col_type in ['open', 'high', 'low', 'close']:
    feature_cols.extend([f'{col_type}_ETHUSDT_lag{i}' for i in range(1, 11)])
for col_type in ['open', 'high', 'low', 'close']:
    feature_cols.extend([f'{col_type}_BTCUSDT_lag{i}' for i in range(1, 11)])
feature_cols.append('hour_of_day')

features = df[feature_cols].values
target = df['target_ETHUSDT'].values

print(f"\nDataset Summary:")
print(f"Total samples: {len(features):,}")
print(f"Number of features: {len(feature_cols)}")
print(f"Feature shape: {features.shape}")
print(f"Target shape: {target.shape}")

=== Loading Data ===

Dataset Summary:
Total samples: 14,630
Number of features: 81
Feature shape: (14630, 81)
Target shape: (14630,)


### 2. Model Training
Trains the model using time series cross-validation and displays performance metrics

In [3]:
# Train model
print("\n=== Training Model ===")
model, metrics, model_info = train_linear_model(features, target, feature_cols)


=== Training Model ===

Top 10 most important features:
                    importance
feature                       
close_ETHUSDT_lag6    0.000452
open_ETHUSDT_lag5     0.000450
open_ETHUSDT_lag3     0.000390
close_ETHUSDT_lag4    0.000385
open_BTCUSDT_lag1     0.000318
close_BTCUSDT_lag2    0.000315
open_BTCUSDT_lag8     0.000298
close_BTCUSDT_lag9    0.000297
close_ETHUSDT_lag8    0.000295
open_ETHUSDT_lag7     0.000286

Model Performance Metrics:
- MSE: 0.00003855 (±0.00001430)
- RMSE: 0.0061 (±0.0013)
- MAE: 0.0039 (±0.0009)
- R²: -0.0521 (±0.0234)
- DirectionalAcc: 0.5434 (±0.0186)


### 3. ONNX Conversion
Converts the trained model to ONNX format for deployment

**Make sure the name of your ONNX model input is `candles`. This is crucial for the evaluation phase!**

In [4]:
import onnxmltools
from skl2onnx.common.data_types import FloatTensorType

def convert_to_onnx(model, input_dim, output_path="eth_ridge_regression.onnx"):
    """Convert trained Ridge Regression model to ONNX format"""
    print(f"\nConverting model to ONNX format: {output_path}")
    
    # Define input type - our features are float32
    initial_type = [('candles', FloatTensorType([None, input_dim]))]
    
    # Convert to ONNX using onnxmltools with skl2onnx backend
    onnx_model = onnxmltools.convert_sklearn(
        model.model,  
        name='RidgePredictor',
        initial_types=initial_type,  
        target_opset=15
    )
    
    # Save the model
    onnxmltools.utils.save_model(onnx_model, output_path)
    print("Conversion complete")
    
    return output_path

In [5]:
# Convert to ONNX
print("\n=== Converting to ONNX ===")
onnx_path = convert_to_onnx(model, len(feature_cols), ONNX_PATH)


=== Converting to ONNX ===

Converting model to ONNX format: eth_ridge_regression.onnx
Conversion complete


### 4. Model Verification
Verifies that the ONNX model produces the same predictions as the original model

In [6]:
print("=== Verifying ONNX Model ===")
try:
    if not os.path.exists(ONNX_PATH):
        raise FileNotFoundError(f"ONNX model not found at {ONNX_PATH}")
        
    session = ort.InferenceSession(ONNX_PATH)
    input_name = session.get_inputs()[0].name
    sample_features = features[:5].astype(np.float32)
    
    # Compare predictions with consistent formatting
    original_pred = model.predict(sample_features)
    onnx_pred = session.run(None, {input_name: sample_features})[0].flatten()
    
    print("\nPrediction Comparison:")
    np.set_printoptions(precision=8, suppress=True)
    print(f"Original model: {original_pred}")
    print(f"ONNX model:     {onnx_pred}")
    
    max_diff = np.max(np.abs(original_pred - onnx_pred))
    print(f"\nMax difference: {max_diff:.8f}")
    
    is_close = np.allclose(original_pred, onnx_pred, rtol=1e-3, atol=1e-4)
    
    if is_close:
        print("\n✅ ONNX model predictions match original model within tolerance")
    else:
        print("\n❌ Warning: ONNX model predictions exceed tolerance threshold!")
except Exception as e:
    print(f"\n❌ Error during verification: {e}")
    raise

=== Verifying ONNX Model ===

Prediction Comparison:
Original model: [ 0.00025253  0.00025462  0.0000617  -0.00022836  0.00005977]
ONNX model:     [ 0.00025253  0.00025462  0.0000617  -0.00022836  0.00005977]

Max difference: 0.00000000

✅ ONNX model predictions match original model within tolerance


## Uploading to the OpenGradient Model Hub

After converting your model to ONNX format, you can upload it to the OpenGradient Model Hub using the OpenGradient SDK.  
For detailed documentation, see the [Model Management Guide](https://docs.opengradient.ai/developers/python_sdk/model_management.html).

Basic workflow:
1. Install the SDK: `pip install opengradient`
2. Initialize the client with your credentials
3. Create a model repository
4. Create a version
5. Upload your ONNX file

### Initialize the SDK

In [ ]:
!pip install opengradient

In [ ]:
import opengradient as og

og.init(private_key="<private_key>", email="<email>", password="<password>")

### Creating & Uploading Your Model

In [8]:
og.create_model(model_name="og-modelthon-eth-ridge-regression", model_desc="example ridge regression model for modelthon")

{'name': 'og-modelthon-eth-ridge-regression', 'versionString': '0.01'}

In [9]:
og.upload(model_path="./eth_ridge_regression.onnx", model_name="og-modelthon-eth-ridge-regression", version='0.01')

{'model_cid': 'QmTqHvTPUDogFHDB1fYLwSFxVX5zFurbnxLkB436n96Qod', 'size': 640}

- to create a new version of your model, invoke the `create_version` function

In [ ]:
versionString = og.create_version(model_name="<model_name>", notes="<notes>")

### Running an Inference on the OpenGradient Network

- For more detailed documentation on running inferences, see the [Verifiable Inference documentation](https://docs.opengradient.ai/developers/python_sdk/inference.html).

**Numerical Precision:**
- Maximum precision: 19 decimal places
- Input values exceeding this precision will cause contract execution to fail

In [10]:
sample_features = features[0:1].astype(np.float32)  # Take first sample, keeping 2D shape
print(f"Input shape: {sample_features.shape}")

# Run inference using the uploaded model
tx_hash, prediction = og.infer(
    model_cid="QmTqHvTPUDogFHDB1fYLwSFxVX5zFurbnxLkB436n96Qod",
    model_input={
        "candles": sample_features.tolist()
    },
    inference_mode=og.InferenceMode.VANILLA
)

print(f"\nLocal Model Prediction: {model.predict(sample_features)}")
print(f"\nPrediction from OpenGradient: {prediction}")
print(f"\nTransaction Hash: {tx_hash}")

Input shape: (1, 81)

Local Model Prediction: [0.00025005]

Prediction from OpenGradient: {'variable': array([[0.00025259]], dtype=float32)}

Transaction Hash: 17d7b33d387a9c2eaa6e0988741219e2bf0636c929c9338913fbafd45fa86736


- check out the transaction on the block explorer:
http://3.145.62.2/tx/0x5972c48b1a5f1320f964d181ba49645fba4fe571e98b05ef62fc2fea617edd3f